# KVecinos en un dataset de detección de spam

Se propone emplear un clasificador basado en distancias sobre el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

## Descarga del dataset Spam
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(4601, 57)


## El clasificador por los vecinos más cercanos

In [2]:
from sklearn.neighbors import KNeighborsClassifier

kv = KNeighborsClassifier()
acc=kv.fit(X_train,y_train).score(X_test,y_test)

print(f'Precisión: {acc:.1%}')

Precisión: 79.5%


**Ejercicio:** Explora el principal parámetros del KNN (n_neighbors) y realiza una búsqueda mediante alguna técnica de optimización ya vista en la práctica anterior

In [13]:
from sklearn.model_selection import GridSearchCV;

G = {"n_neighbors": [1,3,4,5,6,7,8]}

GS = GridSearchCV(KNeighborsClassifier(), G, scoring='accuracy',refit=True, cv=5, verbose = 10)

acc=GS.fit(X_train,y_train).score(X_test,y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV 1/5; 1/7] START n_neighbors=1...............................................
[CV 1/5; 1/7] END ................n_neighbors=1;, score=0.821 total time=   0.1s
[CV 2/5; 1/7] START n_neighbors=1...............................................
[CV 2/5; 1/7] END ................n_neighbors=1;, score=0.818 total time=   0.1s
[CV 3/5; 1/7] START n_neighbors=1...............................................
[CV 3/5; 1/7] END ................n_neighbors=1;, score=0.829 total time=   0.1s
[CV 4/5; 1/7] START n_neighbors=1...............................................
[CV 4/5; 1/7] END ................n_neighbors=1;, score=0.800 total time=   0.1s
[CV 5/5; 1/7] START n_neighbors=1...............................................
[CV 5/5; 1/7] END ................n_neighbors=1;, score=0.810 total time=   0.1s
[CV 1/5; 2/7] START n_neighbors=3...............................................
[CV 1/5; 2/7] END ................n_neighbors=3;,

## Mejoras

La función de distancia empleada por defecto es la distancia euclídea. Dicha distancia requiere un preproceso de las muestras para que tengan una escala similar todas ellas. Además KNN podría beneficiarse de una proyección mediante PCA con el fin de reducir la dimensionalidad.

**Ejercicio:** Implementa un pipeline con la normalización de los datos y un PCA, seguido del KNN. Busca los mejores parámetros. Se podría conseguir una tasa de acierto >90%.


In [4]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures;
from sklearn.model_selection import train_test_split, GridSearchCV;
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA;
from sklearn.pipeline import Pipeline;

scaler = StandardScaler();
pca = PCA();
knn = KNeighborsClassifier();


pipe = Pipeline(steps=[("scaler", scaler),("pca", pca),("knn", knn)])

G = {"pca__n_components": [5,10,15,20,25,30,35],"knn__n_neighbors" : [1,3,4,5,6,7,8]}
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5, verbose = 10)


acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Fitting 5 folds for each of 49 candidates, totalling 245 fits
[CV 1/5; 1/49] START knn__n_neighbors=1, pca__n_components=5....................
[CV 1/5; 1/49] END knn__n_neighbors=1, pca__n_components=5;, score=0.883 total time=   0.1s
[CV 2/5; 1/49] START knn__n_neighbors=1, pca__n_components=5....................
[CV 2/5; 1/49] END knn__n_neighbors=1, pca__n_components=5;, score=0.909 total time=   0.1s
[CV 3/5; 1/49] START knn__n_neighbors=1, pca__n_components=5....................
[CV 3/5; 1/49] END knn__n_neighbors=1, pca__n_components=5;, score=0.882 total time=   0.2s
[CV 4/5; 1/49] START knn__n_neighbors=1, pca__n_components=5....................
[CV 4/5; 1/49] END knn__n_neighbors=1, pca__n_components=5;, score=0.893 total time=   0.2s
[CV 5/5; 1/49] START knn__n_neighbors=1, pca__n_components=5....................
[CV 5/5; 1/49] END knn__n_neighbors=1, pca__n_components=5;, score=0.906 total time=   0.2s
[CV 1/5; 2/49] START knn__n_neighbors=1, pca__n_components=10............

También podríamos probar diferentes funciones de distancia [sklearn distances](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics) a emplear en el parámetro "metric". Así mismo podríamos explorar el parámetro "weights" que pondera el voto de cada vecino de forma diferente según el parámetro escogido.

**Ejercicio:** prueba también diferentes métricas y "weights" junto con todo lo anterior. Emplea el BayessianOpt visto en la práctica anterior.

In [5]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.6 MB/s eta 0:00:00


In [17]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures;
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.decomposition import PCA;
from sklearn.pipeline import Pipeline;

knn = KNeighborsClassifier();

pipe = Pipeline(steps=[("scaler", scaler),("pca", pca),("knn", knn)])


G = {"pca__n_components": Integer(1,57),"knn__n_neighbors" : Integer(1,20), "knn__metric": Categorical(["l1", "l2"]), "knn__weights": Categorical(["uniform","distance"])}
#l1 es distancia manhattan y l2 es distancia euclidea

BS = BayesSearchCV(pipe, G, scoring='accuracy', n_iter=20, refit=True, cv=5)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')

Precisión: 92.9% con OrderedDict([('knn__metric', 'l2'), ('knn__n_neighbors', 20), ('knn__weights', 'distance'), ('pca__n_components', 19)])


## Olivetti Faces

Prueba ahora el clasificador KNN junto con todos los parámetros y preprocesos que creas convenientes sobre el dataset de reconocimiento facial de Olivetti.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split


knn = KNeighborsClassifier();

pipe = Pipeline(steps=[("scaler", scaler),("pca", pca),("knn", knn)])

X, y = fetch_olivetti_faces(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)





GS = GridSearchCV(KNeighborsClassifier(), pipe, G, scoring='accuracy', refit=True, cv=5)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')


In [18]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures;
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.decomposition import PCA;
from sklearn.pipeline import Pipeline;
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split

X, y = fetch_olivetti_faces(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)

knn = KNeighborsClassifier();

pipe = Pipeline(steps=[("scaler", scaler),("pca", pca),("knn", knn)])


G = {"pca__n_components": Integer(1,57),"knn__n_neighbors" : Integer(1,20), "knn__metric": Categorical(["l1", "l2"]), "knn__weights": Categorical(["uniform","distance"])}
#l1 es distancia manhattan y l2 es distancia euclidea

BS = BayesSearchCV(pipe, G, scoring='accuracy', n_iter=20, refit=True, cv=5)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


HTTPError: HTTP Error 403: Forbidden